In [1]:
import json
import openai
import os
import random

import numpy as np
import pandas as pd

Import openai api key from environment variable - here I have it stored as `LOCAL_OPENAI_API_KEY` but you can just paste yours in instead

In [2]:

openai.api_key = os.environ.get('LOCAL_OPENAI_API_KEY')

In [3]:
def query(prompt, **kwargs):
  """
  wrapper for the API to easily parse data
  """
  
  args = {
    "engine":"davinci", # using the original davinci
    "temperature":0, # 0 temperature means it's greedy and gives the same result every time (ish)
    "max_tokens":500, # 500 tokens should be enough
    "stop":"\n\n", # we'll use double newlines to separate the examples
  }
  
  args = {**args, **kwargs}
  
  r = openai.Completion.create(prompt=prompt, **args)["choices"][0]["text"].strip()
  return r


Great, the API key is loaded. Now we can start using the API.

In [4]:
query("q: what is 1+1?\na:")

'2\nq: what is 2+2?\na: 4\nq: what is 3+3?\na: 6\nq: what is 4+4?\na: 8\nq: what is 5+5?\na: 10\nq: what is 6+6?\na: 12\nq: what is 7+7?\na: 14\nq: what is 8+8?\na: 16\nq: what is 9+9?\na: 18\nq: what is 10+10?\na: 20'

This downloads the WIC dataset - you may need to install wget if you don't have it installed https://ports.macports.org/port/wget/

In [5]:
!wget https://pilehvar.github.io/wic/package/WiC_dataset.zip

--2023-02-26 15:59:22--  https://pilehvar.github.io/wic/package/WiC_dataset.zip
Resolving pilehvar.github.io (pilehvar.github.io)... 2606:50c0:8002::153, 2606:50c0:8003::153, 2606:50c0:8000::153, ...
Connecting to pilehvar.github.io (pilehvar.github.io)|2606:50c0:8002::153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 275984 (270K) [application/zip]
Saving to: ‘WiC_dataset.zip’

WiC_dataset.zip     100%[===================>] 269.52K  --.-KB/s    in 0.05s   

2023-02-26 15:59:23 (5.44 MB/s) - ‘WiC_dataset.zip’ saved [275984/275984]



In [6]:
import zipfile
with zipfile.ZipFile("WiC_dataset.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [7]:
train = pd.read_csv("train/train.data.txt", sep='\t', header=None)
train.columns = ["target", "pos", "position", "context-1", "context-2"]
train_gold = pd.read_csv("train/train.gold.txt", sep='\t', header=None)
train_gold.columns = ["label"]
train = pd.concat([train_gold,train], axis=1)


In [15]:
train[train.pos=="V"].tail(18)[:5]


,label,target,pos,position,context-1,context-2
5386,F,admit,V,3-1,The French doors admit onto the yard .,He admitted his errors .
5387,F,exhaust,V,0-4,Exhaust one 's savings .,This kind of work exhausts me .
5389,F,kill,V,0-2,Kill the engine .,She was killed in the collision of three cars .
5390,T,admit,V,1-1,To admit a serious thought into the mind .,She admitted us here .
5394,F,write,V,6-1,How many books did Georges Simenon write ?,Please write to me every week .


We can go and grab the last couple verbs of the train set to use as our few shots. I added some annotation for meaning

In [9]:

fewShotVerb = """Contexts: 'The French doors admit onto the yard .'; 'He admitted his errors .'
Term: 'admit'
Meaning: In the first sentence, 'admit' means to provide passage. In the second, 'admit' means to take responsibility.
They are dissimilar

Contexts: 'The company agrees to meet the cost of any repairs .'; 'Does this paper meet the requirements for the degree ?'
Term: 'meet'
Meaning: In the first sentence, 'meet' means to fulfill. In the second, 'meet' means to take fulfill.
They are similar

Contexts: 'You anger too easily .'; 'He angers easily .'
Term: 'anger'
Meaning: In the first sentence, 'anger' means to enrage. In the second, 'anger' means to enrage.
They are similar

Contexts: 'Exhaust one 's savings .'; 'This kind of work exhausts me .'
Term: 'exhaust'
Meaning: In the first sentence, 'exhaust' means to empty. In the second, 'exhaust' means to tire.
They are dissimilar

Contexts: 'Kill the engine .'; 'She was killed in the collision of three cars .'
Term: 'kill'
Meaning: In the first sentence, 'kill' means to turn off. In the second, 'kill' refers to dying.
They are dissimilar

Contexts: 'To admit a serious thought into the mind .'; 'She admitted us here .'
Term: 'admit'
Meaning: In the first sentence, 'admit' means to allow in. In the second, 'admit' means to allow in.
They are similar

Contexts: 'How many books did Georges Simenon write ?'; 'Please write to me every week .'
Term: 'write'
Meaning: In the first sentence, 'write' means to publish. In the second, 'write' means to communicate with. 
They are dissimilar

Contexts: 'This skill will enable you to find a job on Wall Street .'; 'The rope enables you to secure yourself when you climb the mountain .'
Term: 'enable'
Meaning: In the first sentence, 'enable' means to give an ability. In the second, 'enable' means to give an ability.
They are similar

Contexts: 'The man must answer to his employer for the money entrusted to his care .'; 'She must answer for her actions .'
Term: 'answer'
Meaning: In the first sentence, 'answer' means to take responsibility. In the second, 'answer' means to take responsibility.
They are similar

Contexts: 'Hit the bottle .'; 'He hit a home run .'
Term: 'hit'
Meaning: In the first sentence, 'hit' means to use. In the second, 'hit' means to strike.
They are dissimilar

"""

Likewise we can do it with nouns

In [16]:
fewShotNoun = """Contexts: 'A history of France .'; 'A critical time in the school 's history .'
Term: 'history'
Meaning: In the first sentence, 'history' refers to a record; in the second it means past.
The meanings are dissimilar

Contexts: 'I do it for the fun of it .'; 'He is fun to have around .'
Term: 'fun'
Meaning: In the first sentence, 'fun' means having pleasure; in the second it also means pleasurable.
The meanings are similar

Contexts: 'The rate of production at the factory is skyrocketing .'; 'He works at a great rate .'
Term: 'rate'
Meaning: In the first sentence, 'rate' refers to speed; in the second it also means speed.
The meanings are similar

Contexts: 'Get to the point .'; 'At that point I had to leave .'
Term: 'point'
Meaning: In the first sentence, 'point' refers to a topic; in the second it means time.
The meanings are dissimilar

Contexts: 'Kronas kurss — the exchange rate of the krona .'; 'Grāmata maksā piecas kronas — the book costs five krona .'
Term: 'krona'
Meaning: In the first sentence, 'krona' means money; in the second it also means money.
The meanings are similar

Contexts: 'Armored from head to foot .'; 'The swiftest of foot .'
Term: 'foot'
Meaning: In the first sentence, 'foot' refers to a body part; in the second it means speed.
The meanings are dissimilar

Contexts: 'A patch of clouds .'; 'Patches of thin ice .'
Term: 'patch'
Meaning: In the first sentence, 'patch' means cluster; in the second it also means cluster.
The meanings are similar

Contexts: 'The misery and wretchedness of those slums is intolerable .'; 'She was exhausted by her misery and grief .'
Term: 'misery'
Meaning: In the first sentence, 'misery' means a state of unhappiness; in the second it refers to a feeling of unhappiness.
The meanings are dissimilar

Contexts: 'Women carrying home shopping did n't give me a second glance .'; 'On Saturdays we usually do the shopping .'
Term: 'shopping'
Meaning: In the first sentence, 'shopping' refers to purchases; in the second it means buying things.
The meanings are dissimilar

Contexts: 'While being impulsive can be great for artists , it is not a desirable quality for engineers .'; 'Security , stability , and efficiency are good qualities of an operating system .'
Term: 'quality'
Meaning: In the first sentence, 'quality' means attribute; in the second it also means attribute.
The meanings are similar

"""

In [28]:
def testRow(row):
  prefix = "Does each pair of sentences uses the selected term in a similar sense?\n\n"

  pos = row[1]["pos"]
  examples = fewShotNoun if pos == "N" else fewShotVerb

  context = """Does each pair of sentences uses the selected term in a similar sense?
  
Contexts: '{}'; '{}'
Term: '{}'
Meaning:""".format(row[1]["context-1"], row[1]["context-2"], row[1]["target"])
  return query(prefix + examples + context)


In [23]:
scores = {'V': 0, 'N': 0}
attempted = {'V': 0, 'N': 0}
for row in train.head(10).iterrows():
    print('scores', scores)
    print('attempted', attempted)
    actual = row[1]["label"]
    output = testRow(row)
    pos = row[1]["pos"]
    attempted[pos] += 1

    if actual == "T":
        if output.strip().split()[-1]==("similar"):
            scores[pos] += 1
        
    if actual == "F":
        if output.strip().split()[-1]==("dissimilar"):
            scores[pos] += 1



scores {'V': 0, 'N': 0}
attempted {'V': 0, 'N': 0}
row:  (0, label                                         F
target                                    carry
pos                                           V
position                                    2-1
context-1    You must carry your camping gear .
context-2       Sound carries well over water .
Name: 0, dtype: object)
pos V
scores {'V': 1, 'N': 0}
attempted {'V': 1, 'N': 0}
row:  (1, label                                                       F
target                                                     go
pos                                                         V
position                                                  2-6
context-1      Messages must go through diplomatic channels .
context-2    Do you think the sofa will go through the door ?
Name: 1, dtype: object)
pos V
scores {'V': 2, 'N': 0}
attempted {'V': 2, 'N': 0}
row:  (2, label                                                        F
target                            

In [25]:
scores, attempted

({'V': 5, 'N': 2}, {'V': 7, 'N': 3})

In [26]:
dev = pd.read_csv("dev/dev.data.txt", sep='\t', header=None)
dev.columns = ["target", "pos", "position", "context-1", "context-2"]
dev_gold = pd.read_csv("dev/dev.gold.txt", sep='\t', header=None)
dev_gold.columns = ["label"]
dev = pd.concat([dev_gold,dev], axis=1)


In [32]:
devResults = {}
complete = 0
correct = 0

In [33]:

for row in dev.iterrows():

    if row[0] in devResults:
        continue

    q1 = row[1]["context-1"]
    q2 = row[1]["context-2"]
    target = row[1]["target"]
    actual = row[1]["label"]
    
    pos = row[1]["pos"]

    output = testRow(row)
    
    myResults = {}
    myResults["q1"] = q1
    myResults["q2"] = q2

    myResults["pos"] = row[1]["pos"]

    myResults["target"] = target

    myResults["output"] = output

    myResults["actual"] = actual
    devResults[row[0]] = myResults
    complete +=1
    if actual == "T":
        if output.strip().split()[-1]==("similar"):
            correct += 1
    if actual == "F":
        if output.strip().split()[-1]==("dissimilar"):
            correct += 1

    pct = correct/complete
    pct = round(pct, 2)

    print ("Score: {}, Complete: {} Correct: {} Wrong: {}".format(pct, complete, correct, complete-correct))


Score: 0.0, Complete: 1 Correct: 0 Wrong: 1
Score: 0.0, Complete: 2 Correct: 0 Wrong: 2
Score: 0.33, Complete: 3 Correct: 1 Wrong: 2
Score: 0.25, Complete: 4 Correct: 1 Wrong: 3
Score: 0.2, Complete: 5 Correct: 1 Wrong: 4
Score: 0.33, Complete: 6 Correct: 2 Wrong: 4
Score: 0.43, Complete: 7 Correct: 3 Wrong: 4
Score: 0.38, Complete: 8 Correct: 3 Wrong: 5
Score: 0.44, Complete: 9 Correct: 4 Wrong: 5
Score: 0.5, Complete: 10 Correct: 5 Wrong: 5
Score: 0.55, Complete: 11 Correct: 6 Wrong: 5
Score: 0.58, Complete: 12 Correct: 7 Wrong: 5
Score: 0.62, Complete: 13 Correct: 8 Wrong: 5
Score: 0.64, Complete: 14 Correct: 9 Wrong: 5
Score: 0.67, Complete: 15 Correct: 10 Wrong: 5
Score: 0.69, Complete: 16 Correct: 11 Wrong: 5
Score: 0.71, Complete: 17 Correct: 12 Wrong: 5
Score: 0.72, Complete: 18 Correct: 13 Wrong: 5
Score: 0.74, Complete: 19 Correct: 14 Wrong: 5
Score: 0.75, Complete: 20 Correct: 15 Wrong: 5
Score: 0.71, Complete: 21 Correct: 15 Wrong: 6
Score: 0.68, Complete: 22 Correct: 15 Wr